<a href="https://colab.research.google.com/github/richiedinc/Python/blob/master/MAISON_DNC_Stock_Prediction_Program(%5BDATA_YAHOO%5D).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# This program predicts stock prices by using machine learning models

#Install the dependencies
import  fix_yahoo_finance as fyf
from pandas_datareader import data as pdr
import numpy as np 
import datetime as dt
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [0]:
#Get the stock data
fyf.pdr_override()
df = pdr.get_data_yahoo("TWTR", start ='2000-1-01', end='2019-11-12')
df.head
# Take a look at the data
print(df.head())

[*********************100%***********************]  1 of 1 downloaded
                 Open       High        Low      Close  Adj Close     Volume
Date                                                                        
2013-11-07  45.099998  50.090000  44.000000  44.900002  44.900002  117701600
2013-11-08  45.930000  46.939999  40.689999  41.650002  41.650002   27925300
2013-11-11  40.500000  43.000000  39.400002  42.900002  42.900002   16113900
2013-11-12  43.660000  43.779999  41.830002  41.900002  41.900002    6316700
2013-11-13  41.029999  42.869999  40.759998  42.599998  42.599998    8688300


In [0]:
# Get the Adjusted Close Price
df = df[['Close']]
#Take a look at the new data
print(df.head())

                Close
Date                 
2013-11-07  44.900002
2013-11-08  41.650002
2013-11-11  42.900002
2013-11-12  41.900002
2013-11-13  42.599998


In [0]:
# A variable for predicting 'n' days out into the future
forecast_out = 1 #'n=30' days
#Create another column (the target or dependent variable) shifted 'n' units up
df['Prediction'] = df[['Close']].shift(-forecast_out)
#print the new data set
print(df.tail())

                Close  Prediction
Date                             
2019-11-05  29.809999   29.540001
2019-11-06  29.540001   29.049999
2019-11-07  29.049999   29.209999
2019-11-08  29.209999   29.340000
2019-11-11  29.340000         NaN


In [0]:
### Create the independent data set (X)  #######
# Convert the dataframe to a numpy array
X = np.array(df.drop(['Prediction'],1))

#Remove the last 'n' rows
X = X[:-forecast_out]
print(X)

[[44.900002]
 [41.650002]
 [42.900002]
 ...
 [29.540001]
 [29.049999]
 [29.209999]]


In [0]:
### Create the dependent data set (y)  #####
# Convert the dataframe to a numpy array (All of the values including the NaN's)
y = np.array(df['Prediction'])
# Get all of the y values except the last 'n' rows
y = y[:-forecast_out]
print(y)

[41.650002 42.900002 41.900002 ... 29.049999 29.209999 29.34    ]


In [0]:
# Split the data into 80% training and 20% testing
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [0]:
# Create and train the Support Vector Machine (Regressor)
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_rbf.fit(x_train, y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [0]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction. 
# The best possible score is 1.0
svm_confidence = svr_rbf.score(x_test, y_test)
print("svm confidence: ", svm_confidence)

svm confidence:  0.9851955335602791


In [0]:
# Create and train the Linear Regression  Model
lr = LinearRegression()
# Train the model
lr.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction. 
# The best possible score is 1.0
lr_confidence = lr.score(x_test, y_test)
print("lr confidence: ", lr_confidence)

lr confidence:  0.9859458328379143


In [0]:
# Set x_forecast equal to the last 30 rows of the original data set from Adj. Close column
x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]
print(x_forecast)

[[29.34]]


In [0]:
# Print linear regression model predictions for the next 'n' days
lr_prediction = lr.predict(x_forecast)
print(lr_prediction)

# Print support vector regressor model predictions for the next 'n' days
svm_prediction = svr_rbf.predict(x_forecast)
print(svm_prediction)

[29.34564555]
[29.20293822]
